# Dashboard Acoso/Abuso Sexual en CDMX: dando diseño

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('df_dashing.csv')
df.drop("Unnamed: 0", axis=1, inplace=True)
df.index=pd.to_datetime(df.fecha_hechos)
df = df[(df.index.year >= 2016) & (df.index.year < 2020)]
df=df.drop(['fecha_hechos'],axis=1)

In [3]:
df.loc[df.index.max()]

delito                                    ABS
alcaldia_hechos                    IZTAPALAPA
latitud                               19.3634
longitud                             -99.0142
text               Abuso, 2019-12-31 18:15:00
Name: 2019-12-31 18:15:00, dtype: object

In [4]:
df.loc[df.index.min()]

delito                                    ABS
alcaldia_hechos                ALVARO OBREGON
latitud                               19.3983
longitud                             -99.2049
text               Abuso, 2016-01-01 00:00:00
Name: 2016-01-01 00:00:00, dtype: object

In [5]:
def getCoordinatesHours(Hours, StartDate, EndDate):
    
    return_data = df.loc[StartDate:EndDate]

    #Case 1: neither Hours or Type inserted
    if Hours is None or len(Hours) == 0:
        return return_data
    
    return_string = "return_data["
    for e in Hours:
        if Hours.index(e) is not len(Hours) - 1:
            return_string += "(return_data.index.hour==" + str(int(e)) + ") | "
        else:
            return_string += "(return_data.index.hour==" + str(int(e)) + ")]"
    return eval(return_string)


################################################################################
#2.- Traer por tupo de delito

def getCoordinatesType(Type, Data):

    if Type == 'ACS':
        return Data[Data.delito == 'ACS']
    elif Type == 'ABS':
        return Data[Data.delito == 'ABS']
    else:
        return Data

## DASHING

In [6]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from plotly import graph_objs as go
from plotly.graph_objs import *
from datetime import datetime as dt

In [7]:
app = dash.Dash(
    __name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}]
)

#external_stylesheets = ['https://codepen.io/pollardld/pen/gmjrt.css']

#app = dash.Dash(
#    __name__,
#    external_stylesheets=external_stylesheets
#)

server = app.server


# Plotly mapbox public token
mapbox_access_token = "pk.eyJ1IjoibHVpc2ZlcmxjMTUxNSIsImEiOiJjazUydm50ZjIwMjUxM2pudzFkMDhkaTYwIn0.NQDEMChu5p542DxB9j7d7g"

In [8]:
list_of_locations = {
    "Iztapalapa": {"lat": 19.3421, "lon":-99.0532},
    "Cuahtémoc": {"lat": 19.4356, "lon":-99.1495},
    "Gustavo A. Madero": {"lat": 19.4811, "lon":-99.1065},
    "Venustiano Carranza": {"lat": 19.4306, "lon":-99.095},
    "Álvaro Obregón": {"lat": 19.3603, "lon":-99.2266},
    "Tlalpan": {"lat": 19.2783, "lon":-99.2112},
    "Coyoacán": {"lat": 19.3436, "lon":-99.1562},
    "Benito Juarez": {"lat": 19.3792, "lon":-99.1592},
    "Miguel Hidalgo": {"lat": 19.4305, "lon":-99.2081},
    "Iztacalco": {"lat": 19.3941, "lon":-99.0976},
    "Azcapotzalco": {"lat": 19.4847, "lon":-99.1883},
    "Xochimilco": {"lat": 19.2563, "lon":-99.10},
    "Tlahuac": {"lat": 19.2989, "lon":-99.0434},
    "Cuajimalpa de Morelos": {"lat": 19.3629, "lon":-99.2881},
    "La Magdalena Contreras": {"lat": 19.3145, "lon":-99.24},
    "Milpa Alta": {"lat": 19.1942, "lon":-99.0252}
                  }

In [9]:
app.layout = html.Div(
    #style={'backgroundColor': 'black'},
    children=[
        html.Div(
            className="row",
            children=[
                # Column for user controls
                html.Div(
                    className="four columns div-user-controls",
                    children=[
                        #html.Img(
                        #    className="logo", src=app.get_asset_url("dash-logo-new.png")
                        #),
                        html.H1("ABUSO Y ACOSO SEXUAL - CDMX"),
                        html.P(
                            """Selecciona rangos de fechas y filtra por tipo de delito y horas"""
                        ),
                        html.Div(
                            className="div-for-dropdown",
                            children=[
                                dcc.DatePickerRange(
                                    id="date-picker-range",
                                    min_date_allowed=dt(2016, 1, 1),
                                    max_date_allowed=dt(2019, 10, 30),
                                    start_date_placeholder_text="Start Date",
                                    end_date_placeholder_text="End Date",
                                    start_date='2016-01-01',
                                    end_date='2019-12-31',
                                    display_format="YYYY,MMMM, D",
                                    #style={"border": "0px solid black",
                                    #      "width": "35%",
                                    #      "margin-bottom": "3px"},
                                )
                            ],
                        ),
                        #Dropdowns
                        html.Div(
                            #style={"width": "18%",
                            #      "margin-bottom": "10px"},
                            className="row",
                            children=[
                                html.Div(
                                    className="div-for-dropdown",
                                    children=[
                                        # Checklist for abuso/acoso
                                        dcc.Dropdown(
                                            #optionHeight=100,#para el heigt de las opciones dentro del dropdown
                                            id="type-dropdown",
                                            options=[
                                                {"label":'Abuso Sexual', "value": 'ABS'},
                                                {"label":'Acoso Sexual', "value":"ACS"}
                                            ],
                                            placeholder="Selecciona tipo de delito",
                                            #style={"margin-bottom": "10px"},
                                        )
                                    ],
                                ),
                                html.Div(
                                    className="div-for-dropdown",
                                    #style={"margin-bottom": "3px"},
                                    children=[
                                        # Dropdown for locations on map
                                        dcc.Dropdown(
                                            id="location-dropdown",
                                            options=[
                                                {"label": i, "value": i}
                                                for i in list_of_locations
                                            ],
                                            placeholder="Selecciona una delegación",
                                        )
                                    ],
                                ),
                                html.Div(
                                    className="div-for-dropdown",
                                    #style={"margin-bottom": "3px"},
                                    children=[
                                        # Dropdown to select times
                                        dcc.Dropdown(
                                            id="bar-selector",
                                            options=[
                                                {
                                                    "label": str(n) + ":00",
                                                    "value": str(n),
                                                }
                                                for n in range(24)
                                            ],
                                            multi=True,
                                            placeholder="Selecciona ciertas horas",
                                        )
                                    ],
                                ),
                            ],
                        ),
                        #html.P(id="total-rides"),
                        #html.P(id="total-rides-selection"),
                        #html.P(id="date-value"),
                        #dcc.Markdown(
                        #    children=[
                        #        "Source: [FiveThirtyEight](https://github.com/fivethirtyeight/uber-tlc-foil-response/tree/master/uber-trip-data)"
                        #    ]
                        #),
                    ],
                ),
                # Column for app graphs and plots
                html.Div(
                    className="eight columns div-for-charts bg-grey",
                    children=[
                        dcc.Graph(id="map-graph"),
                        html.Div(
                            className="text-padding",
                            children=[
                                "ShiXiong3"
                            ],
                        ),
                    ],
                ),
            ],
        ),
    ],
)

In [ ]:
@app.callback(
    Output("map-graph", "figure"),
    [
        Input('date-picker-range', 'start_date'),
        Input('date-picker-range', 'end_date'),
        Input('type-dropdown', 'value'),
        Input("bar-selector", "value"),
        Input("location-dropdown", "value"),
    ],
)
def update_graph(StartDate, EndDate, Delito_Type, Hours, selectedLocation):
    zoom = 12.0
    latInitial = 19.4323
    lonInitial = -99.1323
    bearing = 5
    
    #Coordenadas de una alcaldía en específico
    if selectedLocation:
        zoom = 15.0
        latInitial = list_of_locations[selectedLocation]["lat"]
        lonInitial = list_of_locations[selectedLocation]["lon"]
    
    #Coordenadas respecto a horas y rango de fechas
    data_to_show = getCoordinatesHours(Hours, StartDate, EndDate)
    data_to_show2 = getCoordinatesType(Delito_Type, data_to_show)
    
    return go.Figure(
        data=[
            # Data for all rides based on date and time
            Scattermapbox(
                lat=data_to_show2["latitud"],
                lon=data_to_show2["longitud"],
                mode="markers",
                text=data_to_show2.text,
                hoverinfo="text",
                marker=dict(
                    showscale=True,
                    color=np.append(np.insert(data_to_show2.index.hour, 0, 0), 23),
                    opacity=0.6,
                    size=5,
                  # colorscale=[
                  #    [0, "#020f16"],
                  #    [0.04167, "#02141d"],
                  #    [0.0833, "#031924"],
                  #    [0.125, "#272a30"],
                  #    [0.1667, "#343840"],
                  #    [0.2083, "#414650"],
                  #    [0.25, "#414243"],
                  #    [0.292, "#575859"],
                  #    [0.333, "#6c6e6f"],
                  #    [0.375, "#3a1013"],
                  #    [0.4167, "#4d1519"],
                  #    [0.4583, "#601a1f"],
                  #    [0.50, "#770317"],
                  #    [0.54167, "#8b031b"],
                  #    [0.5833, "#c60426"],
                  #    [1.0, "#d90429"],
                  # ],
                    colorbar=dict(
                        title="Hora del día",
                        x=0.93,
                        xpad=0,
                        nticks=24,
                        tickfont=dict(color="black"),
                        titlefont=dict(color="black"),
                        thicknessmode="pixels",
                    ),
                ),
            ),
            # Plot of important locations on the map
            Scattermapbox(
                lat=[list_of_locations[i]["lat"] for i in list_of_locations],
                lon=[list_of_locations[i]["lon"] for i in list_of_locations],
                mode="markers",
                text=[i for i in list_of_locations],
                hoverinfo="text",
                marker=dict(size=10, color="green"),
            ),
        ],
        layout=Layout(
            autosize=False,
            margin=go.layout.Margin(l=0, r=0, t=0, b=0),
            showlegend=False,
            mapbox=dict(
                accesstoken=mapbox_access_token,
                center=dict(lat=latInitial, lon=lonInitial),
                style="dark",
                bearing=bearing,
                zoom=zoom,
            ),
            updatemenus=[
                dict(
                    buttons=(
                        [
                            dict(
                                args=[
                                    {
                                        "mapbox.zoom": 12,
                                        "mapbox.center.lon": "-99.1323",
                                        "mapbox.center.lat": "19.4323",
                                        "mapbox.bearing": 0,
                                        "mapbox.style": "light",
                                    }
                                ],
                                label="Resetear Zoom",
                                method="relayout",
                            )
                        ]
                    ),
                    direction="left",
                    pad={"r": 0, "t": 0, "b": 0, "l": 0},
                    showactive=False,
                    type="buttons",
                    x=0.45,
                    y=0.02,
                    xanchor="left",
                    yanchor="bottom",
                    bgcolor="#000000",
                    borderwidth=1,
                    bordercolor="#f0f0f0",
                    font=dict(color="#FFFFFF"),
                )
            ],
        ),
    )


if __name__ == "__main__":
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Mar/2020 14:03:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:11] "GET /assets/style.css?m=1583784178.933842 HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:11] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:11] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2020 14:03:44] "POST /_dash-upd